In [37]:
import numpy as np
from data_generation_sty import GaussianDataGenerator
from sklearn.linear_model import Lasso

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [38]:
N = 1000
d = 400
m = 40
noise_level_lst = [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
trial_num = 10
cv_num = 5

alpha_lst = [0.0001, 0.001, 0.01, 0.1, 1, 2]

In [39]:
seed = 0
Data_Geneartor = GaussianDataGenerator(N, d, m, 0.3, seed)

true_signal, dictionary, true_indices, true_coefficients, perturbed_signal = Data_Geneartor.shuffle()

In [40]:
test_lasso = Lasso(alpha=0.1, max_iter=10000, tol=0.0001)
test_lasso.fit(dictionary, perturbed_signal)

y_pred = test_lasso.predict(dictionary)
mse = mean_squared_error(perturbed_signal, y_pred)
mse


0.21633251606542714

In [41]:
lasso_cv = GridSearchCV(Lasso(), {'alpha': alpha_lst}, cv=cv_num, scoring='neg_mean_squared_error')

lasso_cv.fit(dictionary, perturbed_signal)

-lasso_cv.cv_results_['mean_test_score']

array([0.18631928, 0.13447204, 0.21703926, 0.21703926, 0.21703926,
       0.21703926])

In [43]:
lasso_cv.best_params_

{'alpha': 0.001}